In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# Any results you write to the current directory are saved as output.

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [2]:
from fastai.vision import *

In [3]:
bs = 32

In [4]:
df_train = pd.read_csv('/kaggle/input/gala-image-tagging/dataset/train.csv')

In [5]:
path= '/kaggle/input/gala-image-tagging/dataset/'

In [6]:
test_df = pd.read_csv('/kaggle/input/gala-image-tagging/dataset/test.csv')

In [7]:
def create_data(bs, Size, train_idx, val_idx):
    tfms = get_transforms(do_flip=True, flip_vert=False, max_rotate=45.0, max_zoom=1.3, max_lighting=0.5)
    
    data = (
        ImageList.from_df(df=df_train, folder='Train Images', path=path)
        .split_by_idxs(train_idx, val_idx)
       .label_from_df(cols='Class')
       .transform(tfms,size=Size)
       .databunch(bs=bs)
       .normalize(imagenet_stats)
    )
    
    
    return data


def add_test_data(data):
    test_data = (ImageList.from_df(df=test_df,folder='Test Images',path=path))
    data.add_test(test_data)
    return data

In [8]:
from sklearn.model_selection import StratifiedKFold
from fastai.metrics import FBeta

skf = StratifiedKFold(n_splits=12, shuffle=True, random_state=42)

fscores = []
test = []

tfms = get_transforms(do_flip=True, flip_vert=False, max_rotate=45.0, max_zoom=1.3, max_lighting=0.5)
tta_params = {'beta':0.12, 'scale':1.0}

for train_index, val_index in skf.split(df_train.index, df_train['Class']):
    data_fold = create_data(32, 128, train_index, val_index)
    data_fold = add_test_data(data_fold)
    learn = cnn_learner(data_fold, models.resnet50, metrics=[FBeta(average='weighted', beta=1)],pretrained=True,path='/kaggle/working/')
    learn.fit_one_cycle(10, 1e-03)
    fscores.append(learn.recorder.metrics)
    pred,y = learn.TTA(ds_type=DatasetType.Test, **tta_params)
    #pred, y = learn.get_preds(DatasetType.Test)
    test.append(pred)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


epoch,train_loss,valid_loss,f_beta,time
0,1.447675,0.770755,0.720718,00:31
1,1.226406,0.721575,0.762145,00:27
2,0.957223,0.581522,0.780321,00:27
3,0.788402,0.533334,0.813824,00:28
4,0.655893,0.512618,0.818783,00:27
5,0.585590,0.483399,0.835403,00:28
6,0.561741,0.489354,0.827697,00:27
7,0.516893,0.471610,0.836231,00:27
8,0.498644,0.469674,0.831809,00:28
9,0.458351,0.464407,0.839304,00:28


epoch,train_loss,valid_loss,f_beta,time
0,1.369325,0.632046,0.770691,00:28
1,1.154487,0.597956,0.794257,00:27
2,0.936870,0.569080,0.794731,00:29
3,0.771106,0.451367,0.834533,00:27
4,0.665583,0.426616,0.856737,00:28
5,0.620337,0.406900,0.844011,00:27


epoch,train_loss,valid_loss,f_beta,time
0,1.405895,0.803498,0.729228,00:31


epoch,train_loss,valid_loss,f_beta,time
0,1.376272,0.868213,0.738282,00:28


epoch,train_loss,valid_loss,f_beta,time
0,1.434623,0.821824,0.713165,00:27


In [9]:
len(test)

12

In [10]:
new_total=0
new_total=new_total+test[10]+test[2] +test[4]+test[5]
new_pred = new_total.argmax(dim=1)
new_sample = test_df.copy()
new_sample['Class'] = new_pred
new_sample.head()

,Image,Class
0,image6245.jpg,2
1,image10409.jpg,2
2,image8692.jpg,2
3,image10517.jpg,2
4,image2580.jpg,0


In [11]:
d={0:'Attire',1:'Decorationandsignage',2:'Food',3:'misc'}
new_sample.replace({'Class':d}, inplace=True)

In [12]:
new_sample.head()

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire


In [13]:
new_sample.to_csv('best_test_score_compact.csv', index=False)

In [14]:
## Score taken 84-89

In [15]:
new_total=0
new_total=new_total+test[0]+test[2] +test[3] +test[4]+test[5] + test[10]+test[11]

In [16]:
new_pred = new_total.argmax(dim=1)
new_sample = test_df.copy()
new_sample['Class'] = new_pred
new_sample.head()

,Image,Class
0,image6245.jpg,2
1,image10409.jpg,2
2,image8692.jpg,2
3,image10517.jpg,2
4,image2580.jpg,0


In [17]:
d={0:'Attire',1:'Decorationandsignage',2:'Food',3:'misc'}
new_sample.replace({'Class':d}, inplace=True)
new_sample.head()

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire


In [18]:
new_sample.to_csv('best_test_score.csv', index=False)

#### Gave an 85.76 f score

In [19]:
total=0
for i in range(12):
    total = total+test[i]
print(total)

tensor([[1.6958e+00, 6.7478e-02, 1.0098e+01, 1.3863e-01],
        [1.2088e+00, 8.5802e-01, 7.2188e+00, 2.7144e+00],
        [9.2347e-01, 5.3641e-02, 9.0175e+00, 2.0054e+00],
        ...,
        [1.0318e-01, 4.9641e-03, 1.2644e-02, 1.1879e+01],
        [2.4692e-02, 1.5772e+00, 7.5909e-03, 1.0391e+01],
        [1.3003e-02, 3.2335e-02, 4.1324e-03, 1.1951e+01]])


In [20]:
pred = total.argmax(dim=1)
pred

tensor([2, 2, 2,  ..., 3, 3, 3])

In [21]:
sample = test_df.copy()
sample['Class'] = pred
sample.head()

,Image,Class
0,image6245.jpg,2
1,image10409.jpg,2
2,image8692.jpg,2
3,image10517.jpg,2
4,image2580.jpg,0


In [22]:
d={0:'Attire',1:'Decorationandsignage',2:'Food',3:'misc'}
sample.replace({'Class':d}, inplace=True)
sample.head()

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire


In [23]:
sample.to_csv('bestest12_cv.csv', index=False)

In [24]:
final = test_df.copy()
final['Class'] = 0
final['Class'] = test[4].argmax(dim=1)
d={0:'Attire',1:'Decorationandsignage',2:'Food',3:'misc'}
final.replace({'Class':d}, inplace=True)
final.head()

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire


In [25]:
final.to_csv('submission89_fbeta.csv', index=False)